<a href="https://colab.research.google.com/github/conhuangn1e/sweetcarna/blob/main/CONNIE_Nucelolar_Compartment_Segmentation_Deep_Learning_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####Author: Hana Jaafari
####Date: January 30, 2025
####Purpose: The objective of this code is to train on high-resolution fluorescent images of NPM1 protein and "learn" the multiple compartments of the nucleolus (the known three or even more). With this segmentation and using fluorescent images of other nucleolar proteins, we can try to learn the concentrations (and consequently other physical properties) of the other nucleolar proteins.

In [1]:
import os
import matplotlib.pylab as plt
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
import torch.nn as nn
import torch.optim as optim
import glob
import time
from skimage import io, transform
from imutils import paths
from tqdm import tqdm

import itertools
import pandas as pd
import torch
from scipy.ndimage import convolve
from scipy.ndimage import convolve
from torch.utils.data import random_split
import skimage as ski
from sklearn.model_selection import train_test_split

np.random.seed(11) # set seed for reproducibility
torch.random.manual_seed(1)

Upload Cell_ROI_Images.zip at this point to the files on the left sidebar

In [3]:
# run this line only once after zip file is uploaded
!unzip Cell_ROI_Images.zip

Archive:  Cell_ROI_Images.zip
   creating: Cell_ROI_Images/
  inflating: __MACOSX/._Cell_ROI_Images  
  inflating: Cell_ROI_Images/ba082356-76fc-409d-9445-4ba55424903a.tiff  
  inflating: __MACOSX/Cell_ROI_Images/._ba082356-76fc-409d-9445-4ba55424903a.tiff  
  inflating: Cell_ROI_Images/._ba082356-76fc-409d-9445-4ba55424903a.tiff  
  inflating: Cell_ROI_Images/058ade94-dbb3-4bde-9539-cf428febbfc1.tiff  
  inflating: __MACOSX/Cell_ROI_Images/._058ade94-dbb3-4bde-9539-cf428febbfc1.tiff  
  inflating: Cell_ROI_Images/._058ade94-dbb3-4bde-9539-cf428febbfc1.tiff  
  inflating: Cell_ROI_Images/fb03d124-a24c-4bce-9591-e5fe11f2bbb3.tiff  
  inflating: __MACOSX/Cell_ROI_Images/._fb03d124-a24c-4bce-9591-e5fe11f2bbb3.tiff  
  inflating: Cell_ROI_Images/._fb03d124-a24c-4bce-9591-e5fe11f2bbb3.tiff  
  inflating: Cell_ROI_Images/20ab104c-dc92-44fa-a67d-c5cf10570b6c.tiff  
  inflating: __MACOSX/Cell_ROI_Images/._20ab104c-dc92-44fa-a67d-c5cf10570b6c.tiff  
  inflating: Cell_ROI_Images/._20ab104c-dc92-

In [4]:
# define the number of channels in the input, number of classes,
# and number of levels in the U-Net model
channel_count = 1 #Since everything is black and white. Channel here refers to colors (like RGB)
class_count = 3 #This will be modified, but let's start off with the canonical three.
# levels_count = 3 #I'm not sure what this refers to

# initialize learning rate, number of epochs to train for, and the
# batch size
learning_rate = 0.001
epoch_count = 1
batch_size = 30

# define the input image dimensions. Images are square
image_dimensions = 256

#Use a GPU if available, otherwise use a CPU
device_type = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Using {device_type} for calculations")

# determine if we will be pinning memory during data loading
pin_memory_status = True if device_type == "cuda" else False

Using cpu for calculations


In [5]:
data_directory_path = 'Cell_ROI_Images'

## Read in and Process Data

#### Using Pytorch

In [6]:
class NPM1_Pol1_Dataset(Dataset):

    def __init__(self, root_dir, transform=None):
        """
        Arguments:
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.root_dir = root_dir
        self.transform = transform

        self.list_of_files=glob.glob(f"{self.root_dir}/*.tiff")

    def __len__(self):
        return len(self.list_of_files)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image = io.imread(self.list_of_files[idx])
        #Only select images that are large enough
        # if image[0].shape[0] >= image_dimensions and image[0].shape[1] >= image_dimensions:
          #NPM1 image is in slice 0, and Pol1 is in slice 1
        sample = {'npm1_image': image[0],
                  'pol1_image': image[1],
                  'file_name':self.list_of_files[idx].split("/")[-1]}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [7]:
#Functions to process images
class StrategicCrop(object):
    """Crop the image in a sample to retain as much intensity.

    Args:
        output_size (tuple or int): Desired output size. If int, square crop
            is made.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            assert len(output_size) == 2
            self.output_size = output_size

    def __call__(self, sample):
        npm1_image, pol1_image, file_names = sample['npm1_image'], sample['pol1_image'], sample['file_name']
        h, w = npm1_image.shape[:2]

        # Ensure the image is large enough for cropping
        if h < self.output_size[0] or w < self.output_size[1]:
            return None

        new_h, new_w = self.output_size
        max_intensity = -np.inf
        best_crop_coords = None

        # Slide a window of crop_size over the entire image
        for i in range(h - new_h + 1):
            for j in range(w - new_w + 1):
                # Define the crop region
                region = npm1_image[i:i + new_h, j:j + new_w]
                # Compute the sum of intensities of the region
                region_intensity = np.sum(region)
                # If this region has a higher intensity sum, store it
                if region_intensity > max_intensity:
                    max_intensity = region_intensity
                    best_crop_coords = (i, j)

        if best_crop_coords is None:
            return None

        i, j = best_crop_coords

        # Apply the same crop to both images
        npm1_crop = npm1_image[i:i + new_h, j:j + new_w]
        pol1_crop = pol1_image[i:i + new_h, j:j + new_w]

        return {'npm1_image': npm1_crop, 'pol1_image': pol1_crop, 'file_name': file_names}

class RandomCrop(object):
    """Crop randomly the image in a sample.

    Args:
        output_size (tuple or int): Desired output size. If int, square crop
            is made.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            assert len(output_size) == 2
            self.output_size = output_size

    def __call__(self, sample):
        npm1_image, pol1_image, file_names = sample['npm1_image'], sample['pol1_image'], sample['file_name']

        h, w = npm1_image.shape[:2]
        new_h, new_w = self.output_size
        try:
          top = np.random.randint(0, h - new_h + 1)
          left = np.random.randint(0, w - new_w + 1)

          npm1_image = npm1_image[top: top + new_h,
                        left: left + new_w]
          pol1_image = pol1_image[top: top + new_h,
                        left: left + new_w]
        except:
          #Print out names of files that cannot be cropped (likely too small)
          print(file_names)

        return {'npm1_image': npm1_image,
                'pol1_image': pol1_image,
                'file_name': file_names}

class Normalize(object):
    """Normalize Images"""
    def __call__(self, sample):
      npm1_image, pol1_image, file_names = sample['npm1_image'], sample['pol1_image'], sample['file_name']

      #Set all values below zero to zero (this might be an abberation from FF correction)
      npm1_image[npm1_image<0]=0
      mean_npm1_value=np.mean(npm1_image)
      std_npm1_value=np.std(npm1_image)
      normalized_npm1_image=(npm1_image-mean_npm1_value)/std_npm1_value

      #Set all values below zero to zero (this might be an abberation from FF correction)
      pol1_image[pol1_image<0]=0
      mean_pol1_value=np.mean(pol1_image)
      std_pol1_value=np.std(pol1_image)
      normalized_pol1_image=(pol1_image-mean_pol1_value)/std_pol1_value

      return {'npm1_image': normalized_npm1_image,
              'pol1_image': normalized_pol1_image,
              'file_name': file_names}


class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""
		#Images need to be reshaped into 3D tensors for UNET. The first dimension
		#is the channel count. This is set to one here as a dummy dimension.
    def __call__(self, sample):
        npm1_image, pol1_image, file_names = sample['npm1_image'], sample['pol1_image'], sample['file_name']
        return {'npm1_image': torch.unsqueeze(torch.from_numpy(npm1_image),0),
                'pol1_image': torch.unsqueeze(torch.from_numpy(pol1_image),0),
                'file_name': file_names}

In [8]:
# remove files that are too small for cropping

allpaths = glob.glob(f"{data_directory_path}/*.tiff")

print("before:", len(allpaths))
for image_file in allpaths:
  image = io.imread(image_file)
  if image[0].shape[0] < image_dimensions or image[0].shape[1] < image_dimensions:
    !rm {image_file}
print("after:", len(glob.glob(f"{data_directory_path}/*.tiff")))

# 754 files --> 703 files

before: 754
after: 703


In [10]:
print("trainingset:", len(os.listdir(f"{data_directory_path}/Training_Set")))
print("validation set:", len(os.listdir(f"{data_directory_path}/Validation_Set")))

trainingset: 217
validation set: 108


In [20]:
#Generate strategic crops of same size of images and convert to tensors
data_transforms = {
    'Training_Set': torchvision.transforms.Compose([
        Normalize(),
        StrategicCrop(image_dimensions), #Need to change this to crop around a region with high intensity
        # torchvision.transforms.RandomHorizontalFlip(),
        ToTensor()
        ]),
    'Validation_Set': torchvision.transforms.Compose([
        Normalize(),
        RandomCrop(image_dimensions),#Need to change this to crop around a region with high intensity
        # torchvision.transforms.RandomHorizontalFlip(),
        ToTensor()
        ])}

#Set up training and validation directories
if len(glob.glob(f'{data_directory_path}/*.tiff'))!= 0: #check if you haven't already split the dataonce before
  train_paths, test_paths = train_test_split(glob.glob(f'{data_directory_path}/*.tiff'), test_size=0.33, random_state=42)
  !mkdir Cell_ROI_Images/Training_Set
  !mkdir Cell_ROI_Images/Validation_Set
  for path in train_paths:
    destination = os.path.join(data_directory_path, "Training_Set", os.path.basename(path))
    !mv {path} {destination}
  for path in test_paths:
    destination = os.path.join(data_directory_path, "Validation_Set", os.path.basename(path))
    !mv {path} {destination}
print("for record keeping: ")
print("Files in training set:", len(os.listdir(f"{data_directory_path}/Training_Set")))
print("Files in validation set:", len(os.listdir(f"{data_directory_path}/Validation_Set")))

#Create dataset
image_datasets = {x: NPM1_Pol1_Dataset(root_dir=os.path.join(data_directory_path,x), transform=data_transforms[x]) for x in ['Training_Set', 'Validation_Set']}

#Upload the transformed dataset to Dataloader iterator
dataloaders = {x: DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True,pin_memory=pin_memory_status,num_workers=os.cpu_count())
              for x in ['Training_Set', 'Validation_Set']}

for record keeping: 
Files in training set: 217
Files in validation set: 108


### Building the U-Net Architecture

In [21]:
class Block(nn.Module):
	def __init__(self, inChannels, outChannels):
		super().__init__()
		# store the convolution and RELU layers
		#The kernal size should not be 1 (third argument) (consistent with U-Net architecture)
    #Using padding=1 to maintain image dimensions
		self.conv1 = nn.Conv2d(inChannels, outChannels, kernel_size=3, padding=1)
		self.relu = nn.ReLU()
		# #The kernal size should not be 1 (third argument) (consistent with U-Net architecture)
    #Using padding=1 to maintain image dimensions
		self.conv2 = nn.Conv2d(outChannels, outChannels, kernel_size=3, padding=1)
    # Pooling acts as a sort of spatial dimensional reduction.
		# self.pool = nn.MaxPool2d(kernel_size=3,stride=1)
	def forward(self, x):
		## apply CONV => RELU => CONV block to the inputs and return it
		# apply CONV => RELU => POOL block to the inputs and return it
    #ReLU will clip the smallest value of inputs to be >=0.
		return self.relu(self.conv2(self.relu(self.conv1(x))))

In [22]:
class Encoder(nn.Module):
	def __init__(self, channels=(1, 16, 32, 64, 128)): #The first argument here is the number of channels in our image.
		super().__init__()
		# store the encoder blocks and maxpooling layer
		self.encBlocks = nn.ModuleList(
			[Block(channels[i], channels[i + 1])
			 	for i in range(len(channels) - 1)])
    #The maxpooling layer will reduce the size of the data by half.
    #This acts as a sort of spatial dimensional reduction.
		# self.pool = nn.MaxPool2d(2)

	def forward(self, x):
		# initialize an empty list to store the intermediate outputs
		blockOutputs = []
		# loop through the encoder blocks
		for block in self.encBlocks:
			# pass the inputs through the current encoder block, store
			# the outputs, and then apply maxpooling on the output
			x = block(x)
			# print("The dimensions after conv pixels: ")
			# print(x.shape)
			blockOutputs.append(x)
			# x = self.pool(x) #Removing pooling for now

		# return the list containing the intermediate outputs
		return blockOutputs

In [23]:
class Decoder(nn.Module):
	def __init__(self, channels=(128, 64, 32, 16)):
		super().__init__()
		# initialize the number of channels, upsampler blocks, and
		# decoder blocks
		self.channels = channels
		self.upconvs = nn.ModuleList(
      #Using padding=1 to maintain image dimensions
			[nn.ConvTranspose2d(channels[i], channels[i + 1], kernel_size=3,padding=1)
			 	for i in range(len(channels) - 1)])
		self.dec_blocks = nn.ModuleList(
			[Block(channels[i], channels[i + 1])
			 	for i in range(len(channels) - 1)])

	def forward(self, x, encFeatures):
		# loop through the number of channels
		for i in range(len(self.channels) - 1):
			# pass the inputs through the upsampler blocks
			x = self.upconvs[i](x)
			# print("The dimensions after upconv pixels: ")
			# print(x.shape)
      #Implementing skip connections now
			# crop the current features from the encoder blocks,
			# concatenate them with the current upsampled features,
			# and pass the concatenated output through the current
			# decoder block
			encFeat = self.crop(encFeatures[i], x)
			x = torch.cat([x, encFeat], dim=1)
			x = self.dec_blocks[i](x)
			# print("The dimensions after deconv channels: ")
			# print(x.shape)
		# return the final decoder output
		return x

	def crop(self, encFeatures, x):
		# grab the dimensions of the inputs, and crop the encoder
		# features to match the dimensions
		(_, _, H, W) = x.shape
		encFeatures = torchvision.transforms.CenterCrop([H, W])(encFeatures)
		# return the cropped features
		return encFeatures

In [24]:
class UNet(nn.Module):
	#For Josh's Mathematica code: encChannels=(1, 64, 128, 256)
	def __init__(self, encChannels=(1, 16, 32, 64, 128, 256), #The first argument here is the number of channels in our image.
		 decChannels=(256, 128, 64, 32, 16),
		 nbClasses=class_count, retainDim=False, #nbClasses refers to the number of segmentation classes possible.
		 outSize=(image_dimensions,  image_dimensions)):
		super().__init__()
		# initialize the encoder and decoder
		self.encoder = Encoder(encChannels)
		self.decoder = Decoder(decChannels)
		# initialize the regression head and store the class variables
		self.head = nn.Conv2d(decChannels[-1], nbClasses, 1)
		self.retainDim = retainDim
		self.outSize = outSize

	def forward(self, x):
			# grab the features from the encoder
			encFeatures = self.encoder(x)
			# pass the encoder features through decoder making sure that
			# their dimensions are suited for concatenation
			decFeatures = self.decoder(encFeatures[::-1][0],
				encFeatures[::-1][1:])
			# pass the decoder features through the regression head to
			# obtain the segmentation mask
			map = self.head(decFeatures)
			# check to see if we are retaining the original output
			# dimensions and if so, then resize the output to match them
			if self.retainDim:
				map = nn.functional.interpolate(map, self.outSize)
			#Normalize all channel values associated with a single pixel to equal one.
			m = nn.Softmax(dim=1)
			normalized_map = m(map)
			# return the segmentation map
			return normalized_map

### Extract Compartment Concentrations and Recapitulate Image of Protein B

In [25]:
def calculate_compartment_concentrations(predicted_compartment_batch_tensor,other_protein_image_batch_tensor):
  #Images are organized as a batch.
  #Work on optimizing these operations
  for (n,predicted_compartment_tensor) in enumerate(predicted_compartment_batch_tensor):
    converted_predicted_compartment_tensor=predicted_compartment_tensor.cpu().clone().detach().numpy()
    #Define the matrix of average compartments values
    averaged_compartment_fraction_matrix=np.zeros((class_count,class_count))
    for i in range(class_count):
      #Extract nxn matrix associated with compartment
      column_selected_compartment_fraction_matrix=np.squeeze(converted_predicted_compartment_tensor[i,:,:])
      for j in range(class_count):
        #Extract nxn matrix associated with compartment
        row_selected_compartment_fraction_matrix=np.squeeze(converted_predicted_compartment_tensor[j,:,:])
        #Multiply the matrices then take average over all pixels
        #Need to do element wise multiplication not matrix multiplication (since each pixel is independent)
        multiplied_matrix=np.multiply(column_selected_compartment_fraction_matrix, row_selected_compartment_fraction_matrix)
        average_pixel_value=np.mean(multiplied_matrix)
        averaged_compartment_fraction_matrix[i,j]=average_pixel_value
    #Take the inverse of the matrix
    inverted_averaged_compartment_fraction_matrix=np.linalg.inv(averaged_compartment_fraction_matrix)
    ###
    #Extract nxn matrix associated with other protein's image matrix.
    other_protein_image_matrix=np.squeeze(other_protein_image_batch_tensor[n,:,:,:].cpu().clone().detach().numpy())
    #Define vector of averaged scaled intensity values
    averaged_scaled_intensity_vector=[]
    for k in range(class_count):
      #Extract nxn matrix associated with compartment
      column_selected_compartment_fraction_matrix=np.squeeze(converted_predicted_compartment_tensor[k,:,:])
      #Multiply the matrices than average over all pixels
      #Need to do element wise multiplication not matrix multiplication (since each pixel is independent)
      multiplied_matrix=np.multiply(column_selected_compartment_fraction_matrix,other_protein_image_matrix)
      average_pixel_value=np.mean(multiplied_matrix)
      averaged_scaled_intensity_vector.append(average_pixel_value)

    averaged_scaled_intensity_vector=np.asarray(averaged_scaled_intensity_vector)

    #Multiply the vector with the inverted matrix
    concentration_vector=np.matmul(averaged_scaled_intensity_vector,inverted_averaged_compartment_fraction_matrix)
    #Setting concentration values to zero and above to be physical.
    concentration_vector[concentration_vector<0]=0
    ###
    #Recapitulate the image of the other protein
    compartment_fraction_matrix=np.squeeze(converted_predicted_compartment_tensor)

    recapitulated_other_protein_image_matrix=np.dot(np.moveaxis(compartment_fraction_matrix, 0, -1),concentration_vector)
    ###
    #Gather calculated values into tensors
    concentration_values_tensor=torch.tensor(concentration_vector,requires_grad=True,dtype=float)
    reformated_concentration_values_tensor=torch.unsqueeze(concentration_values_tensor,0)

    other_protein_image_tensor=torch.tensor(other_protein_image_matrix,requires_grad=True,dtype=float)
    reformated_other_protein_image_tensor=torch.unsqueeze(other_protein_image_tensor,0)

    recapitulated_other_protein_image_tensor=torch.tensor(recapitulated_other_protein_image_matrix,requires_grad=True,dtype=float)
    reformated_recapitulated_other_protein_image_tensor=torch.unsqueeze(recapitulated_other_protein_image_tensor,0)

    #Constructing the output:
    if n==0:
      concentration_values_batch_tensor=reformated_concentration_values_tensor
      new_other_protein_image_batch_tensor=reformated_other_protein_image_tensor
      recapitulated_other_protein_image_batch_tensor=reformated_recapitulated_other_protein_image_tensor
    else:
      concentration_values_batch_tensor=torch.cat((concentration_values_batch_tensor,reformated_concentration_values_tensor),0)
      new_other_protein_image_batch_tensor=torch.cat((new_other_protein_image_batch_tensor,reformated_other_protein_image_tensor),0)
      recapitulated_other_protein_image_batch_tensor=torch.cat((recapitulated_other_protein_image_batch_tensor,reformated_recapitulated_other_protein_image_tensor),0)

  return concentration_values_batch_tensor, new_other_protein_image_batch_tensor, recapitulated_other_protein_image_batch_tensor

### Training the U-Net

In [26]:
# initialize our UNet model
unet = UNet().to(device_type)
# initialize loss function and optimizer
lossFunc = nn.MSELoss()

opt = optim.Adam(unet.parameters(), lr=learning_rate)
# calculate steps per epoch for training and test set
trainSteps = len(image_datasets["Training_Set"]) // batch_size
validSteps = len(image_datasets["Validation_Set"]) // batch_size
# initialize a dictionary to store training history
H = {"train_loss": [], "validation_loss": []}

In [27]:
unet

UNet(
  (encoder): Encoder(
    (encBlocks): ModuleList(
      (0): Block(
        (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (relu): ReLU()
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (1): Block(
        (conv1): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (relu): ReLU()
        (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (2): Block(
        (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (relu): ReLU()
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (3): Block(
        (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (relu): ReLU()
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (4): Block(
        (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1

### Run Over All Data

In [ ]:
# loop over epochs
print("[INFO] training the network...")
startTime = time.time()
for e in tqdm(range(epoch_count)):
	# set the model in training mode
	unet.train()
	# initialize the total training and validation loss
	total_training_loss = 0
	total_validation_loss = 0
	# loop over the training set
	for (i, x) in enumerate(dataloaders["Training_Set"]):
		# send the input to the device
		npm1_image_tensor = x["npm1_image"].to(device_type)
		pol1_image_tensor= x["pol1_image"].to(device_type)

		# perform a forward pass and calculate the training loss
		#Compartments are predicted
		predicted_compartment_tensor = unet(npm1_image_tensor)
		#Extract predicted compartments' concentrations
		concentration_values_batch_tensor, new_other_protein_image_batch_tensor, recapitulated_other_protein_image_batch_tensor=calculate_compartment_concentrations(predicted_compartment_tensor,pol1_image_tensor)

		#"requires_grad=True" is necessary to keep track of the gradients
		#I have to specify the dtype as function was previously producing dtype=double
		training_loss = lossFunc(recapitulated_other_protein_image_batch_tensor,
		                         new_other_protein_image_batch_tensor)

		# first, zero out any previously accumulated gradients, then
		# perform backpropagation, and then update model parameters
		opt.zero_grad()
		training_loss.backward()
		opt.step() #This initiates gradient descent
		# add the loss to the total training loss so far
		total_training_loss += training_loss

		# switch off autograd
		with torch.no_grad():
			# set the model in evaluation mode
			unet.eval()
			# loop over the validation set
			for (j, y) in enumerate(dataloaders["Validation_Set"]):
				# send the input to the device
				npm1_image_tensor = y["npm1_image"].to(device_type)
				pol1_image_tensor= y["pol1_image"].to(device_type)

				# perform a forward pass and calculate the training loss
				#Compartments are predicted
				predicted_compartment_tensor = unet(npm1_image_tensor)
				#Extract predicted compartments' concentrations
				concentration_values_batch_tensor, new_other_protein_image_batch_tensor, recapitulated_other_protein_image_batch_tensor=calculate_compartment_concentrations(predicted_compartment_tensor,pol1_image_tensor)

				#"requires_grad=True" is necessary to keep track of the gradients
				#I have to specify the dtype as function was previously producing dtype=double
				validation_loss = lossFunc(recapitulated_other_protein_image_batch_tensor,
				                           new_other_protein_image_batch_tensor)
				total_validation_loss += validation_loss

		# calculate the average training and validation loss
		average_training_loss = total_training_loss / trainSteps
		average_validation_loss = total_validation_loss / validSteps
		# update our training history
		H["train_loss"].append(average_training_loss.cpu().detach().numpy())
		H["validation_loss"].append(average_validation_loss.cpu().detach().numpy())
		# print the model training and validation information
		print("[INFO] EPOCH: {}/{}".format(e + 1, epoch_count))
		# print("Train loss: {:.6f}, Validation loss: {:.4f}".format(
		# 	average_training_loss, average_validation_loss))
		print("Train loss: {:.6f}, Validation loss: {:.4f}".format(
			training_loss, validation_loss))
	# display the total time needed to perform the training
	endTime = time.time()
	print("[INFO] total time taken to train the model: {:.2f}s".format(
		endTime - startTime))

[INFO] training the network...


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
fig,axes=plt.subplots(1,6,figsize=(20,10))

axes[0].imshow(np.squeeze((npm1_image_tensor.cpu().clone().detach().numpy())[0,:,:]))
axes[1].imshow(np.squeeze((pol1_image_tensor.cpu().clone().detach().numpy())[0,:,:]))
axes[2].imshow(np.squeeze((recapitulated_other_protein_image_batch_tensor.cpu().clone().detach().numpy())[0,:,:]))
axes[3].imshow(np.squeeze(predicted_compartment_tensor[0,0,:,:].cpu().clone().detach().numpy()))
axes[4].imshow(np.squeeze(predicted_compartment_tensor[0,1,:,:].cpu().clone().detach().numpy()))
axes[5].imshow(np.squeeze(predicted_compartment_tensor[0,2,:,:].cpu().clone().detach().numpy()))
plt.tight_layout()

### Testing the Code

In [ ]:
# # loop over epochs
# print("[INFO] training the network...")
# startTime = time.time()
# epoch_count=1
# for e in tqdm(range(epoch_count)):
# 	# set the model in training mode
# 	unet.train()
# 	# initialize the total training and validation loss
# 	total_training_loss = 0
# 	total_validation_loss = 0
# 	# loop over the training set
# 	for (i, x) in enumerate(dataloaders["Training_Set"]):
# 		# send the input to the device
# 		npm1_image_tensor = x["npm1_image"].to(device_type)
# 		#Image needs to be reshaped into a 4D tensor. Pytorch assumes that
# 		#the first dimension is the batch size (it may become lower than the
# 		#set batch size if some values are left over). The second dimension
# 		#is the channels. This is set to one here as a dummy dimension.
# 		#Send this to the transforms section
# 		reshaped_npm1_image_tensor = npm1_image_tensor.reshape((npm1_image_tensor.shape[0], 1, image_dimensions, image_dimensions))

# 		# perform a forward pass and calculate the training loss
# 		#Compartments are predicted
# 		# print(reshaped_npm1_image_tensor.shape)
# 		predicted_compartment_tensor = unet(reshaped_npm1_image_tensor)
# 		# print(predicted_compartment_tensor.shape)
# 		#Extract predicted compartments' concentrations
# 		concentration_vector,other_protein_image_matrix,recapitulated_other_protein_image_matrix=calculate_compartment_concentrations(predicted_compartment_tensor,x["pol1_image"])
# 		# print(other_protein_image_matrix.shape)
# 		# print(recapitulated_other_protein_image_matrix.shape)
# 		print(concentration_vector)
# 		###
# 		fig,axes=plt.subplots(1,6,figsize=(20,10))
# 		axes[0].imshow(np.squeeze(x["npm1_image"].cpu().clone().detach().numpy()))
# 		axes[1].imshow(other_protein_image_matrix)
# 		axes[2].imshow(recapitulated_other_protein_image_matrix)
# 		axes[3].imshow(np.squeeze(predicted_compartment_tensor[0,0,:,:].cpu().clone().detach().numpy()))
# 		axes[4].imshow(np.squeeze(predicted_compartment_tensor[0,1,:,:].cpu().clone().detach().numpy()))
# 		axes[5].imshow(np.squeeze(predicted_compartment_tensor[0,2,:,:].cpu().clone().detach().numpy()))
# 		plt.tight_layout()

# 		if i==0:
# 			break


In [ ]:
sample_1=list(enumerate(dataloaders["Training_Set"]))[1]

In [ ]:
sample_1

In [ ]:
image_dictionary=sample_1[1]
image_dictionary

In [ ]:
sample_list

In [ ]:
sample_list=list(enumerate(dataloaders["Validation_Set"]))

In [ ]:
sample_list[0][1]["npm1_image"][1,:,:,:]

In [ ]:
sample_1=sample_list[0]

image_dictionary=sample_1[1]


sample_npm1 = image_dictionary["npm1_image"].to(device_type)
sample_pol1= image_dictionary["pol1_image"].to(device_type)

reshaped_sample_npm1 = sample_npm1.reshape((sample_npm1.shape[0], 1, image_dimensions, image_dimensions))
predicted_compartment_tensor = unet(reshaped_sample_npm1)

concentration_vector,other_protein_image_matrix,recapitulated_other_protein_image_matrix=calculate_compartment_concentrations(predicted_compartment_tensor,sample_pol1)
print(concentration_vector)
  # fig,axes=plt.subplots(1,3)

  # axes[0].imshow(np.squeeze(image_dictionary["npm1_image"].cpu().clone().detach().numpy()))
  # axes[1].imshow(np.squeeze(image_dictionary["pol1_image"].cpu().clone().detach().numpy()))
  # axes[2].imshow(recapitulated_other_protein_image_matrix)
print(other_protein_image_matrix.shape)
for i in range(25):
  fig,axes=plt.subplots(1,6,figsize=(20,10))
  axes[0].imshow(np.squeeze(image_dictionary["npm1_image"][i,:,:,:].cpu().clone().detach().numpy()))
  axes[1].imshow(np.squeeze(image_dictionary["pol1_image"][i,:,:,:].cpu().clone().detach().numpy()))
  axes[2].imshow(recapitulated_other_protein_image_matrix[i,:,:].cpu().clone().detach().numpy())
  axes[3].imshow(np.squeeze(predicted_compartment_tensor[i,0,:,:].cpu().clone().detach().numpy()))
  axes[4].imshow(np.squeeze(predicted_compartment_tensor[i,1,:,:].cpu().clone().detach().numpy()))
  axes[5].imshow(np.squeeze(predicted_compartment_tensor[i,2,:,:].cpu().clone().detach().numpy()))
  plt.tight_layout()

In [ ]:
sample_npm1 = image_dictionary["npm1_image"].to(device_type)
reshaped_sample_npm1 = sample_npm1.reshape((sample_npm1.shape[0], 1, image_dimensions, image_dimensions))
predicted_compartment_tensor = unet(reshaped_sample_npm1)
predicted_compartment_tensor

In [ ]:
predicted_compartment_tensor.shape

In [ ]:
print(predicted_compartment_tensor[0,:,:,:].max())
print(predicted_compartment_tensor[0,:,:,:].min())

In [ ]:
predicted_compartment_tensor[0,:,:,:].sum()

In [ ]:
fig,axes=plt.subplots(1,5)

axes[0].imshow(np.squeeze(predicted_compartment_tensor[0,0,:,:].cpu().clone().detach().numpy()))
axes[1].imshow(np.squeeze(predicted_compartment_tensor[0,1,:,:].cpu().clone().detach().numpy()))
axes[2].imshow(np.squeeze(predicted_compartment_tensor[0,2,:,:].cpu().clone().detach().numpy()))
axes[3].imshow(np.squeeze(image_dictionary["npm1_image"].cpu().clone().detach().numpy()))
axes[4].imshow(np.squeeze(image_dictionary["pol1_image"].cpu().clone().detach().numpy()))